In [1]:
import numpy as np
import pandas as pd
import random
import scipy.sparse as sp
from collections import defaultdict

In [2]:
max_group_count = 5000

group_member = pd.read_csv(f"./group_users.csv", sep=",")
group_train = pd.read_csv(f"./train_gi.csv", sep=",")
group_val = pd.read_csv(f"./val_gi.csv", sep=",")
group_test = pd.read_csv(f"./test_gi.csv", sep=",")
user_train = pd.read_csv(f"./train_ui.csv", sep=",")
user_val_te = pd.read_csv(f"./val_ui_te.csv", sep=",")
user_val_tr = pd.read_csv(f"./val_ui_tr.csv", sep=",")
user_test_te = pd.read_csv(f"./test_ui_te.csv", sep=",")
user_test_tr = pd.read_csv(f"./test_ui_tr.csv", sep=",")
group_mat = pd.concat([group_train,group_val,group_test],axis=0,ignore_index=True).drop_duplicates()
group_mat.columns = pd.Index([0,1])
group_mat = group_mat[group_mat[0] < max_group_count]
user_mat = pd.concat([user_train,user_val_te,user_val_tr,user_test_te,user_test_tr],axis=0,ignore_index=True).drop_duplicates()
user_mat.columns = pd.Index([0,1])

In [3]:
max_num_member = -1
group_member_dict = defaultdict(list)
for index, row in group_member.iterrows():
    if int(row['group']) >= max_group_count:
        break
    max_num_member = max(max_num_member, int(row['user']))
    group_member_dict[int(row['group'])].append(int(row['user']))
with open(f"./groupMember.txt", mode="w", encoding="UTF8") as f:
    for group, user_list in group_member_dict.items():
        f.write(str(group))
        f.write(" ")
        f.write(",".join([str(member) for member in user_list]))
        f.write("\n")

In [4]:
user_mat

,0,1
0,4413,2422
1,4413,396
2,4413,2748
3,4413,3055
4,4413,1803
...,...,...
508480,8642,7634
508481,8642,13432
508482,8642,1982
508483,8642,22166


In [5]:
group_mat

,0,1
0,0,19900
1,1,6210
2,2,6428
3,3,3720
4,4,5033
...,...,...
15294,4998,17323
15295,4998,19946
15296,4998,10456
15297,4998,7718


In [6]:
num_users, num_items, num_groups = (max(user_mat[0].values.max(),max_num_member) + 1), (max(user_mat[1].values.max(),group_mat[1].values.max()) + 1), (group_mat[0].values.max() + 1)
num_users, num_items, num_groups

(8643, 25081, 5000)

In [7]:
user_train_item = []
user_test_pos_neg = []
for u in range(num_users):
    i_list = user_mat[user_mat[0] == u][1].tolist()
    # 随机删除一个交互作为测试集
    if len(i_list) > 1:
        test_i = np.random.choice(i_list)
        i_list.remove(test_i)
        assert len(i_list) == len(user_mat[user_mat[0] == u][1].tolist()) - 1
        for train_i in i_list:
            user_train_item.append([u, train_i])
        
        temp_count = 0
        each_user_test_neg_item = []
        while True:
            neg = np.random.choice(num_items)
            if  neg != test_i and neg not in each_user_test_neg_item:
                each_user_test_neg_item.append(neg)
                temp_count+=1
            if temp_count>=100:
                break
        
        user_test_pos_neg.append([u, test_i, each_user_test_neg_item])
    elif len(i_list) == 1:
        test_i = np.random.choice(i_list)
        temp_count = 0
        each_user_test_neg_item = []
        while True:
            neg = np.random.choice(num_items)
            if  neg != test_i and neg not in each_user_test_neg_item:
                each_user_test_neg_item.append(neg)
                temp_count+=1
            if temp_count>=100:
                break
        user_test_pos_neg.append([u, test_i, each_user_test_neg_item])
    
with open(f"./userRatingTrain.txt", mode="w", encoding="UTF8") as f:
    for each in user_train_item:
        f.write(str(each[0]))
        f.write(" ")
        f.write(str(each[1]))
        f.write("\n")
with open(f"./userRatingTest.txt", mode="w", encoding="UTF8") as f:
    for each in user_test_pos_neg:
        f.write(str(each[0]))
        f.write(" ")
        f.write(str(each[1]))
        f.write("\n")
with open(f"./userRatingNegative.txt", mode="w", encoding="UTF8") as f:
    for each in user_test_pos_neg:
        f.write(f"({each[0]},{each[1]})")
        f.write(" ")
        f.write(" ".join([str(v) for v in each[2]]))
        f.write("\n")

In [ ]:
group_train_item = []
group_test_pos_neg = []
for g in range(num_groups):
    i_list = group_mat[group_mat[0] == g][1].tolist()
    # 随机删除一个交互作为测试集
    if len(i_list) > 1:
        test_i = np.random.choice(i_list)
        i_list.remove(test_i)
        assert len(i_list) == len(group_mat[group_mat[0] == g][1].tolist()) - 1
        for train_i in i_list:
            group_train_item.append([g, train_i])
        
        temp_count = 0
        each_group_test_neg_item = []
        while True:
            neg = np.random.choice(num_items)
            if  neg != test_i and neg not in each_group_test_neg_item:
                each_group_test_neg_item.append(neg)
                temp_count+=1
            if temp_count>=100:
                break
        
        group_test_pos_neg.append([g, test_i, each_group_test_neg_item])
    elif len(i_list) == 1:
        test_i = np.random.choice(i_list)
        temp_count = 0
        each_group_test_neg_item = []
        while True:
            neg = np.random.choice(num_items)
            if  neg != test_i and neg not in each_group_test_neg_item:
                each_group_test_neg_item.append(neg)
                temp_count+=1
            if temp_count>=100:
                break
        group_test_pos_neg.append([g, test_i, each_group_test_neg_item])
    
with open(f"./groupRatingTrain.txt", mode="w", encoding="UTF8") as f:
    for each in group_train_item:
        f.write(str(each[0]))
        f.write(" ")
        f.write(str(each[1]))
        f.write("\n")
with open(f"./groupRatingTest.txt", mode="w", encoding="UTF8") as f:
    for each in group_test_pos_neg:
        f.write(str(each[0]))
        f.write(" ")
        f.write(str(each[1]))
        f.write("\n")
with open(f"./groupRatingNegative.txt", mode="w", encoding="UTF8") as f:
    for each in group_test_pos_neg:
        f.write(f"({each[0]},{each[1]})")
        f.write(" ")
        f.write(" ".join([str(v) for v in each[2]]))
        f.write("\n")